In [1]:
%matplotlib qt
import seaborn as sbn
import numpy as np
import matplotlib.pyplot as plt 
from scipy import stats
import tifffile
import os
import sklearn
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV
import imagej
from jnius import autoclass

In [2]:
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()

The JVM is already running.


NameError: name 'ij' is not defined

In [ ]:
#ijgui = autoclass('ij.Gui')
dir(autoclass)

In [ ]:
ij.py.run_macro("""run("mask w wand")""")

In [ ]:
#idx = [55374, 55376, 55380, 55385, 55400, 55406]
ii = 55375
fn = f'/NSLS2/xf18id1/users/2020Q1/ENYUAN_Proposal_305836/masked_spec_{ii}.tif'
img = tifffile.imread(fn)

img1 = img[(img!=img.min())&(img!=img.max())]
print(img.min(), img.max())
print(img1.min(), img1.max())

img = img[(img!=img.min())&(img!=img.max())].flatten()
val = np.unique(img)
print(np.abs(val[1:]-val[:-1]).min())
print(img.shape)
print(val.shape)

"""
grid = GridSearchCV(KernelDensity(atol=1e-5, rtol=1e-5, kernel="gaussian"),
                    {'bandwidth': np.linspace(1e-6, 1e-4, 50)},
                    cv=5, n_jobs=36) # 20-fold cross-validation
grid.fit(img[:, None])
print(grid.best_params_)

kde1 = grid.best_estimator_
kde1.set_params(**{"bandwidth":np.abs(val[1:]-val[:-1]).min()/1.5})
"""

kde1 =  KernelDensity(bandwidth=np.abs(val[1:]-val[:-1]).min()/1.5, 
                      algorithm="kd_tree", kernel='gaussian', 
                      metric='euclidean', atol=1e-5, rtol=1e-5, 
                      breadth_first=True, leaf_size=40, metric_params=None).fit(img[:, None])
x_grid1 = np.linspace(val.min(), val.max(), 500)
pdf1 = np.exp(kde1.score_samples(x_grid1[:, None]))
fig, ax = plt.subplots()
ax.plot(x_grid1, pdf1, linewidth=3, alpha=0.5, label=str(ii))
ax.hist(img, val.shape[0], fc='gray', histtype='stepfilled', alpha=0.3, density=True)
#ax.hist(img, val.shape[0], fc='gray', histtype='stepfilled', alpha=0.3)
ax.legend(loc='upper left')
ax.set_xlim(val.min()-0.001, val.max()+0.001);

"""
plt.figure(1)
sbn.distplot(img, kde=True, hist=False,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
plt.figure(2)
sbn.distplot(img, kde=False, hist=True,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
plt.figure(3)
sbn.distplot(img, kde=True, hist=True,
             kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
             norm_hist=False, label=str(ii))
"""

In [ ]:
dir(kde1)

In [ ]:
print(pdf1.sum()*(val.max()-val.min()+0.002)/2000)
print((np.abs(val[1:]-val[:-1]).min()/1.5))
kde1.get_params()

In [ ]:
idx = [55374, 55376, 55380, 55385, 55400, 55406]
#ii = 55406
for ii in idx:
    fn = f'/NSLS2/xf18id1/users/2020Q1/ENYUAN_Proposal_305836/masked_spec_{ii}.tif'
    img = tifffile.imread(fn)
    img = img[(img!=img.min())&(img!=img.max())].flatten()
    val = np.unique(img)
    print(np.abs(val[1:]-val[:-1]).min())
    print(img.shape)
    print(val.shape)

    grid = GridSearchCV(KernelDensity(atol=1e-5, rtol=1e-5, kernel="gaussian"),
                        {'bandwidth': np.linspace(1e-6, 1e-4, 50)},
                        cv=5, n_jobs=36) # 20-fold cross-validation
    grid.fit(img[:, None])
    print(grid.best_params_)
    
    plt.figure(1)
    sbn.distplot(img, kde=True, hist=False,
                 kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
                 norm_hist=False, label=str(ii))
    plt.figure(2)
    sbn.distplot(img, kde=False, hist=True,
                 kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
                 norm_hist=False, label=str(ii))
    plt.figure(3)
    sbn.distplot(img, kde=True, hist=True,
                 kde_kws={"bw":np.abs(val[1:]-val[:-1]).min(), "shade":True},
                 norm_hist=False, label=str(ii))

#sbn.distplot(img, kde=True, 
#             kde_kws={"bw":10*grid.best_params_['bandwidth']},
#            norm_hist=False)

"""

xmin = (img[(img!=0)&(img!=8.358)].flatten().min()+0.003)*1e5
xmax = (img[(img!=0)&(img!=8.358)].flatten().max()-0.003)*1e5
x_grid = np.linspace(xmin, xmax, 10*int(np.ceil(npnt/grid.best_params_['bandwidth'])))

kde = grid.best_estimator_
pdf = np.exp(kde.score_samples(x_grid[:, None]))

npnt = np.unique((img[(img!=0)&(img!=8.358)].flatten()*1e5)).shape[0]
xmin = (img[(img!=0)&(img!=8.358)].flatten().min()+0.003)*1e5
xmax = (img[(img!=0)&(img!=8.358)].flatten().max()-0.003)*1e5
x_grid = np.linspace(xmin, xmax, 10*int(np.ceil(npnt/grid.best_params_['bandwidth'])))
#x_grid = np.linspace(xmin, xmax, npnt)
kde = grid.best_estimator_
pdf = np.exp(kde.score_samples(x_grid[:, None]))
#pdf1 = kde.score_samples(x_grid[:, None])

fig, ax = plt.subplots()
ax.plot(x_grid, pdf, linewidth=3, alpha=0.5, label='bw=%.6f' % kde.bandwidth)
ax.hist(img[(img!=0)&(img!=8.358)].flatten()*1e5, 67, fc='red', histtype='stepfilled', alpha=0.5, density=True)
ax.legend(loc='upper left')
ax.set_xlim(xmin, xmax);
for cnt, ele in enumerate(zip(pdf1, pdf)):
    print(cnt, ele)
    
"""

In [ ]:

grid = GridSearchCV(KernelDensity(atol=1e-5, rtol=1e-5),
                    {'bandwidth': np.linspace(1e-1, 1e0, 11)},
                    cv=20, n_jobs=36) # 20-fold cross-validation

ii = 55374
fn = f'/NSLS2/xf18id1/users/2020Q1/ENYUAN_Proposal_305836/masked_spec_{ii}.tif'
img = tifffile.imread(fn)
#img[(img!=0)&(img!=8.358)]

grid.fit((img[(img!=0)&(img!=8.358)].flatten()*1e5)[:, None])
print(grid.best_params_)
print((img[(img!=0)&(img!=8.358)].flatten()*1e5).unique().shape)


"""
grid = GridSearchCV(KernelDensity(rtol=1e-5),
                    {'bandwidth': np.linspace(1e-1, 1e-0, 11)},
                    cv=20, n_jobs=36) # 20-fold cross-validation
idx = [55374, 55376, 55380, 55385, 55400, 55406]

for ii in idx:
    fn = f'/NSLS2/xf18id1/users/2020Q1/ENYUAN_Proposal_305836/masked_spec_{ii}.tif'
    img = tifffile.imread(fn)
    #img[(img!=0)&(img!=8.358)]
    
    grid.fit((img[(img!=0)&(img!=8.358)].flatten()*1e5)[:, None])
    print(grid.best_params_)

    #plt.figure(ii)
    #sbn.kdeplot(img[(img!=0)&(img!=8.358)], shade=True, bw=10e-5)
    
    #sbn.distplot(img[(img!=0)&(img!=8.358)], kde=True, norm_hist=True, fit=stats.laplace)
    #a, b = stats.laplace.fit(img[(img!=0)&(img!=8.358)])
    #print(a, b)
    #plt.plot(np.linspace(8.343, 8.358, 151, endpoint=True), 
    #         stats.laplace.pdf(np.linspace(8.343, 8.358, 151, endpoint=True), a, b))
"""


In [ ]:
print(np.unique((img[(img!=0)&(img!=8.358)].flatten()*1e5)).shape)

In [ ]:
npnt = np.unique((img[(img!=0)&(img!=8.358)].flatten()*1e5)).shape[0]
xmin = (img[(img!=0)&(img!=8.358)].flatten().min()+0.003)*1e5
xmax = (img[(img!=0)&(img!=8.358)].flatten().max()-0.003)*1e5
x_grid = np.linspace(xmin, xmax, 10*int(np.ceil(npnt/grid.best_params_['bandwidth'])))
#x_grid = np.linspace(xmin, xmax, npnt)
kde = grid.best_estimator_
pdf = np.exp(kde.score_samples(x_grid[:, None]))
#pdf1 = kde.score_samples(x_grid[:, None])

fig, ax = plt.subplots()
ax.plot(x_grid, pdf, linewidth=3, alpha=0.5, label='bw=%.6f' % kde.bandwidth)
ax.hist(img[(img!=0)&(img!=8.358)].flatten()*1e5, 67, fc='red', histtype='stepfilled', alpha=0.5, density=True)
ax.legend(loc='upper left')
ax.set_xlim(xmin, xmax);
for cnt, ele in enumerate(zip(pdf1, pdf)):
    print(cnt, ele)

In [ ]:
#help(sbn.kdeplot)
#dir(sklearn)
#help(kde.get_params)
#help(grid.fit)

#help(kde.score_samples)

print(kde.get_params())

In [ ]:
from scipy.stats.distributions import norm
#x_grid1 = np.linspace(-4.5, 3.5, 20000)
xmin = -20
xmax = 20
x_grid1 = np.linspace(xmin, xmax, 200)

# Draw points from a bimodal distribution in 1D
np.random.seed(0)
x = np.concatenate([norm(-1, 1.).rvs(40),
                    norm(1, 0.3).rvs(500)])
pdf_true = (0.8 * norm(-1, 1).pdf(x_grid1) +
            0.2 * norm(1, 0.3).pdf(x_grid1))

grid1 = GridSearchCV(KernelDensity(),
                    {'bandwidth': np.linspace(0.1, 1.0, 30)},
                    cv=20, n_jobs=36) # 20-fold cross-validation
grid1.fit(x[:, None])
print(grid1.best_params_)

kde1 = grid1.best_estimator_
pdf1 = np.exp(kde1.score_samples(x_grid1[:, None]))

fig, ax = plt.subplots()
ax.plot(x_grid1, pdf1, linewidth=3, alpha=0.5, label='bw=%.2f' % kde1.bandwidth)
ax.hist(x, 30, fc='gray', histtype='stepfilled', alpha=0.3, density=True)
ax.legend(loc='upper left')
#ax.set_xlim(-4.5, 3.5);
ax.set_xlim(xmin, xmax);

In [ ]:
pdf1 = np.exp(kde1.score_samples(x_grid1[:, None]))
print(pdf1.sum()*(xmax-xmin)/200.)